# Benchmark Taylor green

In [1]:
#https://jsdokken.com/dolfinx-tutorial/chapter2/navierstokes.html#variational-formulation

from dolfinx import mesh, fem, io
import ufl
from mpi4py import MPI
from petsc4py import PETSc
import numpy as np

T = 0.1
num_steps = 100
n_cells = 32

In [2]:
domain = mesh.create_unit_cube(MPI.COMM_WORLD, n_cells, n_cells, n_cells) 
velocity_function_space = fem.functionspace(domain, ("Lagrange", 2, (3,))) 
pressure_function_space = fem.functionspace(domain, ("Lagrange", 1))

In [3]:
f = fem.Constant(domain, PETSc.ScalarType((0,0,0)))
dt = fem.Constant(domain, T/num_steps)
mu = fem.Constant(domain, PETSc.ScalarType(1/50)) # Re = 50
rho = fem.Constant(domain, PETSc.ScalarType(1))

In [4]:
# solución analitica sacada de: https://www.ljll.fr/~frey/papers/Navier-Stokes/Ethier%20C.R.,%20Steinman%20D.A.,%20Exact%20fully%203d%20Navier-Stokes%20solutions%20for%20benchmarking.pdf
a = np.pi/4
d = np.pi/2

def u_analytic(x, y, z, t):
    return np.vstack((-a*(np.exp(a*x)*np.sin(a*y + d*z) + np.exp(a*z)*np.cos(a*x + d*y))*np.exp(-1*d*d*t),
              -a*(np.exp(a*y)*np.sin(a*z + d*x) + np.exp(a*x)*np.cos(a*y + d*z))*np.exp(-1*d*d*t),
              -a*(np.exp(a*z)*np.sin(a*x + d*y) + np.exp(a*y)*np.cos(a*z + d*x))*np.exp(-1*d*d*t)
                     ))

def p_analytic(x, y, z, t):
    return -1*a*a*(1/2)*(np.exp(2*a*x) + np.exp(2*a*y) + np.exp(2*a*z) + 2*np.sin(a*x + d*y)*np.cos(a*z + d*x)*np.exp(a*y + a*z) \
                        + 2*np.sin(a*y + d*z)*np.cos(a*x + d*y)*np.exp(a*z + a*x) + 2*np.sin(a*z + d*x)*np.cos(a*y + d*z)*np.exp(a*x + a*y)) \
            *np.exp(-2*d*d*t)

u_sol_analytic = fem.Function(velocity_function_space)
P_sol_analytic = fem.Function(pressure_function_space)

In [5]:
u = ufl.TrialFunction(velocity_function_space)
p = ufl.TrialFunction(pressure_function_space)

v = ufl.TestFunction(velocity_function_space)
q = ufl.TestFunction(pressure_function_space)

u_sol = fem.Function(velocity_function_space) # guarda la solucion del tiempo actual
u_prev = fem.Function(velocity_function_space) # guarda la solucion del tiempo anterior
p_sol = fem.Function(pressure_function_space) 
p_prev = fem.Function(pressure_function_space)

# interpolar la condicion inicial
u_prev.interpolate(lambda x: u_analytic(*x, 0))
p_prev.interpolate(lambda x: p_analytic(*x, 0))

In [6]:
# Dirichlet BC en todas las fronteras, dada por la solución analítica
domain.topology.create_connectivity(domain.topology.dim-1, domain.topology.dim)
boundary_facets = mesh.exterior_facet_indices(domain.topology) # acá hay un tema con los procesos y es que devuelve los indices locales (del proceso)
dofs_boundary_u = fem.locate_dofs_topological(velocity_function_space, domain.topology.dim-1, boundary_facets)
dofs_boundary_p = fem.locate_dofs_topological(pressure_function_space, domain.topology.dim-1, boundary_facets)

u_bc = fem.Function(velocity_function_space)
p_bc = fem.Function(pressure_function_space)
bc_u  = fem.dirichletbc(u_bc, dofs_boundary_u)
bc_p  = fem.dirichletbc(p_bc, dofs_boundary_p)

In [7]:
from ufl import FacetNormal, dx, ds, dot, inner, sym, nabla_grad, Identity, lhs, rhs, div

u_midpoint = 0.5*(u_prev + u)
n = FacetNormal(domain)

def epsilon(u):
    return sym(nabla_grad(u))

def sigma(u, p):
    return 2*mu*epsilon(u) - p*Identity(len(u))

# step 1
form1 = rho*dot((u - u_prev) / dt, v)*dx \
      + rho*dot(dot(u_prev, nabla_grad(u_prev)), v)*dx \
      + inner(sigma(u_midpoint, p_prev), epsilon(v))*dx \
      + dot(p_prev*n, v)*ds - dot(mu*nabla_grad(u_midpoint)*n, v)*ds \
      - dot(f, v)*dx
bilinear1 = lhs(form1)
linear1 = rhs(form1)

# step 2
form2 = dot(nabla_grad(p), nabla_grad(q))*dx \
      - dot(nabla_grad(p_prev), nabla_grad(q))*dx \
      + (rho/dt)*div(u_sol)*q*dx
bilinear2 = lhs(form2)
linear2 = rhs(form2)

# step 3
form3 = rho*dot((u - u_sol), v)*dx \
      + dt*dot(nabla_grad(p_sol - p_prev), v)*dx
bilinear3 = lhs(form3)
linear3 = rhs(form3)

In [8]:
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells

tree = bb_tree(domain, domain.geometry.dim)
points = np.array([[0, 0, 0], [1, 0, 0], [1, 1, 1]])
cell_candidates = compute_collisions_points(tree, points)
colliding_cells = compute_colliding_cells(domain, cell_candidates, points)
cells = [] 
cells.append(colliding_cells.links(0)[0])
cells.append(colliding_cells.links(1)[0])
cells.append(colliding_cells.links(2)[0])

In [18]:
from dolfinx.fem.petsc import LinearProblem
from dolfinx.fem import assemble_scalar
from tqdm.notebook import tqdm
from datetime import date
from dolfinx.geometry import bb_tree, compute_collisions_points, compute_colliding_cells

t = 0
i = 0
folder = datetime.now(tz=timezone(-timedelta(hours=5))).isoformat(timespec='seconds')
u_file = io.VTXWriter(domain.comm, f"{folder}/velocity.bp", u_sol)
p_file = io.VTXWriter(domain.comm, f"{folder}/pressure.bp", p_sol)
u_file.write(t)
p_file.write(t)

comm = u_sol.function_space.mesh.comm
progress = tqdm(desc="Resolviendo navier-stokes", total=num_steps)

with open(f"{date.today()}/error.txt", "w") as error_log:
    for n in range(num_steps):
        progress.update()
        t += dt.value
        i += 1
    
        u_bc.interpolate(lambda x: u_analytic(*x, t))
        p_bc.interpolate(lambda x: p_analytic(*x, t))
        
        problem1 = LinearProblem(bilinear1, linear1, [bc_u], u_sol)
        problem1.solve()
    
        problem2 = LinearProblem(bilinear2, linear2, [bc_p], p_sol)
        problem2.solve()
    
        problem3 = LinearProblem(bilinear3, linear3, [bc_u], u_sol)
        problem3.solve()
    
        u_file.write(t)
        p_file.write(t)

        u_prev.x.array[:] = u_sol.x.array
        p_prev.x.array[:] = p_sol.x.array
    
        # error relativo: |u_sol - u_analitica| / |u_analitica|
        u_sol_analytic.interpolate(lambda x: u_analytic(*x, t))

        #if t == 0.000:
        print("t=",t)
        for (j, point) in enumerate(points):
            print("aprox", u_sol.eval(point, cells[j]))
            print("aprox2", u_prev.eval(point, cells[j]))
            print("inter", u_sol_analytic.eval(point, cells[j]))
            print("exact", u_analytic(*point,t).flatten())
            print("error", np.sum(u_sol.eval(point, cells[j]) -u_analytic(*point,t).flatten()))
        
        # con norma L_inf
        #error = np.abs(u_sol_analytic.x.array - u_sol.x.array).max()/np.abs(u_sol_analytic.x.array).max()
        # con norma L_2
        error_abs_integral = fem.form(inner(u_sol_analytic - u_sol, u_sol_analytic - u_sol) * dx)
        error_abs = np.sqrt(comm.allreduce(assemble_scalar(error_abs_integral), op=MPI.SUM))
        norm_u_analytic_integral = fem.form(inner(u_sol_analytic, u_sol_analytic) * dx)
        norm_u_analytic = np.sqrt(comm.allreduce(assemble_scalar(norm_u_analytic_integral), op=MPI.SUM))
        error = error_abs / norm_u_analytic
        error_log.write('t = %.3f: error = %.3g' % (t, error) + "\n")

progress.close()
u_file.close()
p_file.close()

Resolviendo navier-stokes:   0%|          | 0/100 [00:00<?, ?it/s]

t= 0.001
aprox [-0.78346457 -0.78346457 -0.78346457]
aprox2 [-0.78346457 -0.78346457 -0.78346457]
inter [-0.78346266 -0.78346266 -0.78346266]
exact [-0.78346266 -0.78346266 -0.78346266]
error -5.721311830408915e-06
aprox [-0.55399311 -2.50182177 -0.55399311]
aprox2 [-0.55399311 -2.50182177 -0.55399311]
inter [-0.55399176 -2.50181568 -0.55399176]
exact [-0.55399176 -2.50181568 -0.55399176]
error -8.786969240226483e-06
aprox [-1.93739057e-16 -5.15861791e-16 -5.26059773e-16]
aprox2 [-1.93739057e-16 -5.15861791e-16 -5.26059773e-16]
inter [-1.93977356e-16 -5.16064527e-16 -5.26126738e-16]
exact [-1.73963657e-16 -1.73963657e-16 -1.73963657e-16]
error -7.137696504686228e-16
t= 0.002
aprox [-0.78153384 -0.78153384 -0.78153384]
aprox2 [-0.78153384 -0.78153384 -0.78153384]
inter [-0.78153193 -0.78153193 -0.78153193]
exact [-0.78153193 -0.78153193 -0.78153193]
error -5.727543278766234e-06
aprox [-0.55262787 -2.49565641 -0.55262787]
aprox2 [-0.55262787 -2.49565641 -0.55262787]
inter [-0.55262652 -2